In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import re
from graphframes import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import *

In [7]:
from IPython.display import display, HTML

In [2]:
sqlContext = SQLContext(sc)

In [ ]:
path_to_graph='./9_11_edgelist.txt'
def articulation(path_to_graph):
    with open(path_to_graph) as f:
        lines=f.readlines()
    #converting file to nodes and edges in rdd
    rdd_lines= sc.parallelize(lines)
    rdd_nodes = rdd_lines.flatMap(lambda x: re.split(',',x.strip())).distinct().map(lambda x: (x,))
    rdd_edges = rdd_lines.map(lambda x: tuple(re.split(',',x.strip())))
    #print rdd_edges.collect()
    #print rdd_nodes.collect()
    
    #converting rdd to dataframe
    df_nodes = sqlContext.createDataFrame(rdd_nodes, ['id'])
    df_edges = sqlContext.createDataFrame(rdd_edges, ['src', 'dst'])
    
    #Initializing graphframe
    graph = GraphFrame(df_nodes, df_edges)
    graph_connectedcomponents_len = len(graph.connectedComponents().groupBy('component').count().collect())
    
    nodes = graph.vertices
    edges = graph.edges

    articulation_table_list = []
    for n in nodes.flatMap(lambda x:x).collect():
        print("1")
        nodes_nremoved = nodes.filter(nodes['id'] != n)
        print("2")
        edges_nremoved = edges.filter(edges['src'] != n).filter(edges['dst'] != n)
        print("3")
        graph_nremoved = GraphFrame(nodes_nremoved,edges_nremoved)
        print("4")
        graph_nremoved_conncomps_len = len(graph_nremoved.connectedComponents().groupBy('component').count().collect())
        print("5")
        articulation_table_list.append([n, graph_nremoved_conncomps_len>graph_connectedcomponents_len])
    articulation_table = sqlContext.createDataFrame(articulation_table_list, ['id', 'articulation'])
    return articulation_table
       
        
answer = articulation(path_to_graph)   
answer

1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4


In [18]:
answer[answer['articulation']==True].head(50)

[]